# Testing File
## this file not use for training the model

In [4]:
TF_ENABLE_ONEDNN_OPTS=0

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [6]:
data = pd.read_csv('Dataset/combine.csv')

In [7]:
data

In [8]:
X = data.drop(['Height cm (H)','Width cm (W)','Lower Angle (L)','Upper Angle (U)','Internal Angle (Right,2nd)','Leave Model'],axis=1)
y = data['Leave Model']

In [9]:
plt.scatter(X['Ratio (H/W)'], X['Ratio(L/U)'], c=y)
plt.colorbar()
plt.show()

In [25]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# x_scaled = scaler.fit_transform(X)
# X = pd.DataFrame(x_scaled, columns=['Ratio (H/W)','Ratio(L/U)','Ratio(L/I)'])

In [29]:
X

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
y_test

In [9]:
def filter_model_info(model_info):
    filtered_info = []
    for layer_info in model_info:
        layer = {
            "class_name" : layer_info['class_name'],
            "batch_input_shape" : layer_info['batch_input_shape'],
            "units": layer_info['config'].get('units'),
            "activation" : layer_info['config'].get('activation')
        }
    
def save_perfomance_metrics(file_path,model, history):
    with open(file_path,'a') as file:
        model_info = {
            "model_architecture" : {
                "name": model['config']['name'],
                "layers": model['config']['layers']
            }
        }

In [10]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(16, activation='relu', input_shape=(3,)),
#     tf.keras.layers.Dense(3, activation='softmax')
# ])
# 
# model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
#              optimizer = tf.keras.optimizers.Adam(learning_rate=0.005),
#              metrics = ["accuracy"])
# 
# epoch_number = 10
# history = model.fit(X_train, y_train, epochs=epoch_number)

In [19]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(16,activation='relu',input_shape=(3,)))
    
    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    for i in range(hp.Int('num_layers',1,20)):
        model.add(tf.keras.layers.Dense(units=hp_units,activation='relu'))
        
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
                  metrics=['accuracy'])
    
    return model

In [65]:
from keras_tuner import RandomSearch
tuner = RandomSearch(build_model, objective='val_accuracy',max_trials=20, executions_per_trial=3, directory='my_dir1',project_name='Tree Classifier')

In [66]:
tuner.search_space_summary()

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [67]:
tuner.search(X_train, y_train,
             epochs=500,
             validation_data=(X_test, y_test),
             callbacks=[early_stopping])

In [72]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# You can now evaluate the best model on your test data, for example:
# test_loss, test_acc = best_model.evaluate(X_test, y_test)

In [79]:
# Evaluate the best model on the validation data
val_loss, val_accuracy = best_model.evaluate(X_train, y_train)

# Print the validation loss and accuracy
print('Validation loss: ', val_loss)
print('Validation accuracy: ', val_accuracy)

In [74]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [77]:
print(best_hps.values)

In [68]:
import json

# Load the data from the trial.json file
with open('my_dir1/Tree Classifier/trial_0/trial.json', 'r') as f:
    data = json.load(f)

In [41]:
# Extract the values and steps for each metric
train_accuracy = data['metrics']['metrics']['accuracy']['observations'][0]['value'][0]
train_accuracy_step = data['metrics']['metrics']['accuracy']['observations'][0]['step']

val_loss = data['metrics']['metrics']['val_loss']['observations'][0]['value'][0]
val_loss_step = data['metrics']['metrics']['val_loss']['observations'][0]['step']

val_accuracy = data['metrics']['metrics']['val_accuracy']['observations'][0]['value'][0]
val_accuracy_step = data['metrics']['metrics']['val_accuracy']['observations'][0]['step']

In [44]:
val_loss

In [45]:
# Plot the data
plt.figure(figsize=(10, 6))

plt.plot([train_accuracy_step], [train_accuracy], marker='o', label='Training Accuracy')
plt.plot([val_loss_step], [val_loss], marker='o', label='Validation Loss')
plt.plot([val_accuracy_step], [val_accuracy], marker='o', label='Validation Accuracy')

plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()

plt.show()

In [69]:
import glob

files = glob.glob('my_dir1/Tree Classifier/**/trial*.json', recursive=True)

steps = []
val_losses = []
val_accuracies = []
train_accuracies = []
trial_numbers = []

for i,file in enumerate(files):
    with open(file,'r') as f:
        data = json.load(f)
        
        step = data['metrics']['metrics']['val_loss']['observations'][0]['step']
        val_loss = data['metrics']['metrics']['val_loss']['observations'][0]['value'][0]
        val_accuracy = data['metrics']['metrics']['val_accuracy']['observations'][0]['value'][0]
        train_accuracy = data['metrics']['metrics']['accuracy']['observations'][0]['value'][0]
        
        steps.append(step)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        train_accuracies.append(train_accuracy)
        trial_numbers.append(f'Trial {i+1}') 

In [46]:
import plotly.graph_objects as go

In [71]:
trace1 = go.Scatter(x=steps, y=train_accuracies, mode='lines+markers', name='Training Accuracy',text=trial_numbers)
trace2 = go.Scatter(x=steps, y=val_losses, mode='lines+markers', name='Validation Loss',text=trial_numbers)
trace3 = go.Scatter(x=steps, y=val_accuracies, mode='lines+markers', name='Validation Accuracy',text=trial_numbers)

# Create a layout
layout = go.Layout(title='Analysis', xaxis=dict(title='Epoch'), yaxis=dict(title='Value'))

# Create a Figure and add the traces
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

# Show the figure
fig.show()

In [47]:
# Create a trace for each metric
trace1 = go.Scatter(x=[train_accuracy_step], y=[train_accuracy], mode='markers', name='Training Accuracy')
trace2 = go.Scatter(x=[val_loss_step], y=[val_loss], mode='markers', name='Validation Loss')
trace3 = go.Scatter(x=[val_accuracy_step], y=[val_accuracy], mode='markers', name='Validation Accuracy')

# Create a layout
layout = go.Layout(title='Metrics vs Epoch', xaxis=dict(title='Epoch'), yaxis=dict(title='Value'))

# Create a Figure and add the traces
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

# Show the figure
fig.show()

In [48]:
import json

# a = json.loads(model.to_json())
# a['config']['layers']

In [ ]:
epoch_number = 10
history = model.fit(X_train, y_train, epochs=epoch_number)

In [11]:
y_pred = model.predict(X_test)
y_pred[:5]

In [14]:
y_pred = np.argmax(y_pred, axis=1)
y_pred[:5]

In [15]:
model.evaluate(X_test, y_test)

In [16]:
pd.DataFrame(history.history).plot()